In [100]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()x
import torch

# 创建数据

In [101]:
data = np.array([
        ["病人","头疼","肌肉疼","体温","流感"]
        ,["e1",'是', '是', '正常', '否']
        ,["e2",'是', '是', '高', '是']
        ,["e3",'是', '是', '很高', '是']
        ,["e4",'否', '是', '正常', '否']
        ,["e5",'否', '否', '高', '否']
        ,["e6",'否', '是', '很高', '是']])

# 分组统计

In [102]:
def prob(mat):
    info = mat[1:,1:-1]
    m,n = info.shape
    D = {mat[0,-1]:mat[1:,-1]}
    e = mat[0][1:-1]
    return {k:{
        x:(v==x).dot(np.ones(v.shape[-1]))/v.shape[-1] 
            for x in np.unique(v)} 
            for k,v in zip(e,info.T)},D,e

prob(data)

({'头疼': {'否': 0.5, '是': 0.5},
  '肌肉疼': {'否': 0.16666666666666666, '是': 0.8333333333333334},
  '体温': {'很高': 0.3333333333333333,
   '正常': 0.3333333333333333,
   '高': 0.3333333333333333}},
 {'流感': array(['否', '是', '是', '否', '否', '是'], dtype='<U3')},
 array(['头疼', '肌肉疼', '体温'], dtype='<U3'))

# 值摘取

In [103]:
def filter_(data,leve1_keyword=None,leve2_keyword=None):
    info , D,e = prob(data)
    if leve2_keyword != None:
        return info[leve1_keyword][leve2_keyword]
    return info[leve1_keyword]
filter_(data,"头疼","是")

0.5

# 计算函数

In [104]:

def mutual_info(px,py):
    return (px*py)*((px*py)/np.log2(px*py))
mutual_info(filter_(data,"头疼","是"),filter_(data,"头疼","否"))

-0.03125

In [105]:
mutual_info(filter_(data,"头疼","是"),filter_(data,"头疼","否"))

-0.03125

# 逻辑运行

In [106]:
X1 = np.array([0.1 for x in range(1200)])
Y1 = np.array([0.3 for x in range(1200)])
X1,Y1

(array([0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]),
 array([0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3]))

In [107]:
data = np.array([['病人', '头疼', '肌肉疼', '体温', '流感'],
       ['e1', '是', '是', '正常', '否'],
       ['e2', '是', '是', '高', '是'],
       ['e3', '是', '是', '很高', '是'],
       ['e4', '否', '是', '正常', '否'],
       ['e5', '否', '否', '高', '否'],
       ['e6', '否', '是', '很高', '是']], dtype='<U3')
data

array([['病人', '头疼', '肌肉疼', '体温', '流感'],
       ['e1', '是', '是', '正常', '否'],
       ['e2', '是', '是', '高', '是'],
       ['e3', '是', '是', '很高', '是'],
       ['e4', '否', '是', '正常', '否'],
       ['e5', '否', '否', '高', '否'],
       ['e6', '否', '是', '很高', '是']], dtype='<U3')

In [108]:
X = np.array([1,0,1,1,0])
Y = np.array([1,1,1,0,0])

In [109]:
def checkType(X):
    if isinstance(X,(np.ndarray,list,tuple)):
        return torch.FloatTensor(X)
    elif isinstance(X,(torch.TensorType,torch.FloatType)):
        return X
    else:
        print("Type Error")

In [110]:

def P(X,symbol,x):
    n = X.size
    Lambda = "{}{}{}".format(X,symbol,x)
    return (X==x).dot(np.ones(n))/n

# 信息熵

In [111]:
def H(X):
    temp = np.array([-P(X,'==',x)*np.log2(P(X,'==',x)) for x in np.unique(X)])
    return temp.sum()
H(X)

0.9709505944546686

# 联合熵

# $$ H(X,Y) = - \sum_{x \in X} \sum_{y \in Y} P(x,y)logP(x,y)  $$

In [112]:
def HXY(X,Y):
    Xn,Yn = X.size,Y.size
    init_XY = np.vstack((X,Y)).T
    m,n = init_XY.shape
    enumerate_ = np.array([(x,y) for x in np.unique(X) for y in np.unique(Y)])
    temp = np.array([((init_XY==e).dot(np.ones(n))==n).dot(np.ones(m))/m for e in enumerate_])
    return (-temp*np.log2(temp)).sum()
HXY(X,Y)

1.9219280948873623

In [113]:
def joint_entropy(X,Y):
    m = X.shape[0]
    comb = lambda SET : np.array([[i,j] for i in SET[::-1] for j in SET])
    element = comb(np.union1d(np.unique(X),np.unique(Y)))
    PXY = np.array([((np.c_[X,Y]==e).dot(np.ones(e.size))==e.size).dot(np.ones(m))/m for e in element]) 
    return (-PXY*np.log2(PXY)).dot(np.ones(PXY.size))
joint_entropy(X,Y)

1.9219280948873623

In [114]:
def TorchJointEntropy(X,Y):
    m = X.size
    comb = lambda SET : np.array([[i,j] for i in SET[::-1] for j in SET])
    element = checkType(comb(np.union1d(np.unique(X),np.unique(Y))))
    XY = torch.cat([checkType(X)[:,None],checkType(Y)[:,None]],1)
    PXY = torch.tensor([(((XY == e).float()).mm(torch.ones((2,1)))==2).sum().float()/m for e in element])
    return (-PXY*torch.log2(PXY)).sum()
TorchJointEntropy(X,Y)

tensor(1.9219)

# 互信息

In [115]:
def I(X,Y):
    return H(X)+H(Y)-HXY(X,Y)
I(X,Y)

0.01997309402197489

In [116]:
I(X1,Y1)

0.0

# $$ I(X;Y) = \sum_{x \in X}\sum_{y \in Y} P(x,y)log \frac{P(x,y)}{P(x)P(y)}$$

### 先算联合概率

In [117]:
m = X.shape[0]
comb = lambda SET : np.array([[i,j] for i in SET[::-1] for j in SET])
element = comb(np.union1d(np.unique(X),np.unique(Y)))
PXY = np.array([((np.c_[X,Y]==e).dot(np.ones(e.size))==e.size).dot(np.ones(m))/m for e in element])
PXY

array([0.2, 0.4, 0.2, 0.2])

### 计算边缘概率

In [136]:
M = X.size+Y.size
Pxy = np.array([(np.r_[Y,X]==x).sum()/M for x in np.unique(X) ])
Pxy.dot(Pxy)

0.52

### 求解互信息

In [138]:
pxpy = Xpxy*Ypxy
tmp = 0
for pxy in PXY:
    tmp += (pxy*np.log2(pxy/(Pxy**2))).sum()
    print(tmp)

-0.10521376233351759
0.48435871299944727
0.3791449506659297
0.2739311883324121


# 条件熵

In [120]:
def HX_Y(X,Y):
    return H(X)-I(X,Y)
HX_Y(X,Y)

0.9509775004326937

https://blog.csdn.net/tangxianyu/article/details/105759989
http://www.fuzihao.org/blog/2015/01/17/%E4%BA%92%E4%BF%A1%E6%81%AF%E7%9A%84%E7%90%86%E8%A7%A3/